# AICore-Bridge

> Bridge between Stactics AICore framework and Wodan/Conan processor modules

In [ ]:
#| default_exp aicorebridge

In [ ]:
#| export

import typing
import logging
import inspect
import traceback
import datetime
import json
import pandas as pd
import numpy as np

from fastcore.basics import patch_to, patch

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
syslog = logging.getLogger(__name__)

In [ ]:
#| export
print(f"loading {__name__}")

In [ ]:
#| export
class AICoreModule(): pass

The custom AICoreModule `__init__` takes the processor as our custom parameter as well as the parameters required by AICore; `assets_dir` and `save_dir`. Any additional parameters are accepted as generic `*args` and `**kwargs`.

In [ ]:
#| export
@patch
def __init__(self:AICoreModule, 
             processor:typing.Callable, # data processing function
             save_dir:str, # path where the module can keep files 
             assets_dir:str,
             *args, **kwargs):
    
    self.directory_args = dict(
        save_dir=save_dir,
        assets_dir=assets_dir
    )
    
    self._init_processor(processor)

    self.init_args = args
    self.init_kwargs = {k:v for k,v in kwargs.items() if v is not None}

    self.init_time = datetime.datetime.now(datetime.UTC)


The private `_init_processor` gets information about the call signature for later reuse in `infer`

In [ ]:
#| export
@patch
def _init_processor(
        self:AICoreModule, 
        processor:typing.Callable):
    """Initializes processor related variables on self"""
    
    self.processor = processor
    self.processor_signature = inspect.signature(self.processor)
    self.processor_params = dict(self.processor_signature.parameters)
    self.return_param = self.processor_params.pop('return', None)
    self.data_param, *self.call_params = list(self.processor_params.keys())


In [ ]:
#| export
@patch
def call_processor(self:AICoreModule, calldata, **callargs):
    return self.processor(calldata, **callargs)


In [ ]:
#| export
@patch
def infer(self:AICoreModule, data:dict, *_, **kwargs):
    msg=[]
    try:

        msg=[
            f"{self.processor.__name__}({self.processor_signature})",
            f"init_args: {self.init_args}",
            f"init_kwargs: {self.init_kwargs}",
            f"init time: {self.init_time.isoformat()}"
        ]

        lastSeen = kwargs.pop('lastSeen', False)
        recordformat = kwargs.pop('format', "records").lower()
        reversed = kwargs.pop('reversed', False)
        timezone = kwargs.get('timezone', 'UTC')
        msg.append(f"lastSeen: {lastSeen}, recordformat: {recordformat}, timezone: {timezone}")

        calldata = self.get_call_data(
            data, 
            recordformat=recordformat,
            timezone=timezone,
            reversed=reversed)
        
        msg.append(f"calldata shape: {calldata.shape}")

        callargs = self.get_callargs(**kwargs, **self.directory_args)

        for arg, val in callargs.items():
            msg.append(f"{arg}: {val}")
            
        result = self.call_processor(calldata, **callargs)
        msg.append(f"result shape: {result.shape}")

        return {
            'msg':msg,
            'data': self.rewrite_data(
                result if not lastSeen else result[-1:],
                recordformat=recordformat,
                timezone=timezone,
                reversed=reversed)
        }

    except Exception as err:
        msg.append(''.join(traceback.format_exception(None, err, err.__traceback__)))
        syslog.exception(f"Exception {str(err)} in infer()")
        return {
            'msg': msg,
            'data': []
        }


In [ ]:
#| export
@patch
def get_callargs(self:AICoreModule, **kwargs):
    "Get arguments for the processor call"

    # Remove null / None values
    kwargs = {k:v for k,v in kwargs.items() if v is not None}
    
    metadata = kwargs.pop('metadata', {}) # TODO: historic metadata

    return {
        K:self.processor_signature.parameters[K].annotation(kwargs.get(K,metadata.get(K, self.init_kwargs.get(K, self.processor_signature.parameters[K].default))))
        for K in self.call_params
    }


In [ ]:
#| export
@patch
def get_call_data(
        self:AICoreModule, 
        data:dict, 
        recordformat='records', 
        timezone='UTC', 
        reversed=False):
    
    "Convert data to the processor signature"

    orient = recordformat.lower()
    assert orient in ['records', 'table']
    #assert False, "started work on format"
    
    if orient == 'records':
        df = pd.DataFrame.from_records(data)
        time_column = [C for C in df.columns if C.lower() in ['datetimemeasure', 'time']][0]

    elif orient == 'table':
        time_column = data['schema']['primaryKey'][0]
        df = pd.DataFrame.from_dict(data['data']).set_index(data['schema']['primaryKey'])
        df.index.name = 'time'

    df.columns = [C.lower() for C in df.columns]
    time_column = [C for C in df.columns if C in ['datetimemeasure', 'time']][0]
    df[time_column] = pd.to_datetime(df[time_column],utc=True,format='ISO8601')
    df.set_index(time_column, inplace=True)
    #df.index = pd.DatetimeIndex(df.index).round('ms')
    
    df.index.name = 'time'

    if reversed:
        df = df[::-1]

    if not df.index.tz:
        df.index = df.index.tz_localize('UTC').tz_convert(timezone)
    elif str(df.index.tz) != timezone:
        df.index = df.index.tz_convert(timezone)

    if self.processor_params[self.data_param].annotation == pd.DataFrame:
        return df
    elif len(df.columns) > 1:
        df.index = (df.index - datetime.datetime(1970,1,1, tzinfo=datetime.timezone.utc)) / datetime.timedelta(seconds=1)
        return df.to_records(index=True)
    else:
        df.index = (df.index - datetime.datetime(1970,1,1, tzinfo=datetime.timezone.utc)) / datetime.timedelta(seconds=1)
        return df.reset_index().to_numpy()
        

In [ ]:
#| export
@patch
def rewrite_data(
        self:AICoreModule, 
        data:typing.Union[pd.DataFrame, pd.Series, dict], 
        recordformat:str='split', 
        timezone:str='UTC', 
        reversed:bool=False):
    
    "Rewrite data to dictionary matching the requested recordformat"

    orient = recordformat.lower()

    if orient == 'split-index':
        orient = 'split'

    normalized_data = self.convert_to_dataframe(data, timezone=timezone)

    if isinstance(normalized_data.index, pd.DatetimeIndex):
        normalized_data.index = normalized_data.index.map(lambda x: x.isoformat())
    
    normalized_data.sort_index(ascending=not bool(reversed), inplace=True)

    if orient == 'records':
        records = normalized_data.reset_index().to_dict(orient='records')
    else:
        records =  normalized_data.to_dict(orient=orient)
    

    if normalized_data.isna().any(axis=None):
        return [ {k:v for k,v in m.items() if pd.notnull(v)} for m in records]
    else:
        return records

    

In [ ]:
#| export
@patch
def convert_to_dataframe(
        self:AICoreModule, 
        adata:typing.Union[pd.DataFrame, pd.Series, dict, np.ndarray, np.recarray], 
        timezone='UTC', 
        columnnames=None):
    """Convert various data formats to standardized timeseries DataFrame"""

    if isinstance(adata, pd.DataFrame):
        df = adata
    elif isinstance(adata, pd.Series):
        df = pd.DataFrame(adata)

    elif isinstance(adata, dict):
        # dict/mapping of individual timeseries
        df = pd.DataFrame({
            C:pd.Series(data=A[:,1], index=pd.DatetimeIndex(A[:,0]*1e9)) if isinstance(A, np.ndarray) else A
            for C,A in adata.items()
        })

    elif adata.dtype.names is not None:
        # structured or recarray
        df = pd.DataFrame(
            data=adata.view(dtype=np.float64).reshape(adata.shape[0],len(adata.dtype))[:,range(1,len(adata.dtype))],
            index=pd.DatetimeIndex(adata.view(dtype=np.float64).reshape(adata.shape[0],len(adata.dtype))[:,0] * 1e9),
            columns=adata.dtype.names[1:]
        )

    else:
        if adata.shape[0] > 0:
            if adata.shape[1]>2:
                columns=[f"value_{str(i+1)}" for i in range(adata.shape[1]-1)] if not columnnames else [f"{str(i)}" for i in columnnames[1:]]
            else:
                columns=['value']

            df = pd.DataFrame(
                data=adata[:, 1:],
                index=pd.DatetimeIndex(adata[:,0]*1e9),
                columns=columns
            )
        else:
            return pd.DataFrame()

    if isinstance(df.index, pd.DatetimeIndex):
        df.index.name = 'time'
        if not df.index.tz:
            df.index = df.index.tz_localize('UTC').tz_convert(timezone)
        elif str(df.index.tz) != timezone:
            df.index = df.index.tz_convert(timezone)

    return df

### References

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()